In [1]:
import os

In [2]:
%pwd

'h:\\Chicken-disease-prediction-MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Chicken-disease-prediction-MLOPS'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self)->PrepareCallbacksConfig:
        
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([model_ckpt_dir, config.tensorboard_root_log_dir])
        
        prepare_callback_config=PrepareCallbacksConfig(
            root_dir=self.config.artifacts_root,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )
        
        return prepare_callback_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [10]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}",)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
        )
        
    def get_tb_ckpt_callbacks(self):
        return[self._create_tb_callbacks,
               self._create_checkpoint_callbacks
               ]

In [12]:
try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callback_config()
    prepare_callback=PrepareCallback(prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-10-23 00:22:44,854: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-10-23 00:22:44,884: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-23 00:22:44,889: INFO: common: created directory at: artifacts]
[2024-10-23 00:22:44,893: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-10-23 00:22:44,896: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
